In [71]:
## !rm -rf /content/DataLake
## !rm -f /content/DataLake.zip

In [72]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Bronze_Mubawab").getOrCreate()

spark


In [73]:
from google.colab import files
uploaded = files.upload()

Saving DataLake.zip to DataLake.zip


In [74]:
import zipfile
import os

zip_path = "/content/DataLake.zip"
extract_path = "/content/"

with zipfile.ZipFile(zip_path, 'r') as z:
    z.extractall(extract_path)

os.listdir("/content")

['.config',
 'DataLake.zip',
 '.ipynb_checkpoints',
 'DataLake',
 'drop',
 'sample_data']

In [76]:
from pyspark.sql import functions as F

raw_base = "/content/DataLake/raw"

# --- Mubawab ventes (PARQUET) ---
mubawab_ventes_df = (
    spark.read
    .parquet(f"{raw_base}/mubawab/ventes/*/*/*/*")   # YYYY/MM/DD/HHMMSS
)

# --- Mubawab locations (PARQUET) ---
mubawab_locations_df = (
    spark.read
    .parquet(f"{raw_base}/mubawab/locations/*/*/*/*")
)

# Union comme avant
df = mubawab_ventes_df.unionByName(mubawab_locations_df)

In [77]:
df.printSchema()

root
 |-- attributes: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- breadcrumbs: string (nullable = true)
 |-- breadcrumbs_list: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- category_label: string (nullable = true)
 |-- description: string (nullable = true)
 |-- equipments: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: string (nullable = true)
 |-- images: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- location: string (nullable = true)
 |-- price_text: string (nullable = true)
 |-- published_date: string (nullable = true)
 |-- scraping_time: string (nullable = true)
 |-- seller_is_store: boolean (nullable = true)
 |-- seller_name: string (nullable = true)
 |-- seller_url: string (nullable = true)
 |-- title: string (nullable = true)
 |-- url: string (nullable = true)
 |-- source_site: string (nullable = true)
 |-- offre: string (nullabl

In [78]:
df.count()

66

In [82]:
df.limit(2).toPandas()

,attributes,breadcrumbs,breadcrumbs_list,category_label,description,equipments,id,images,location,price_text,published_date,scraping_time,seller_is_store,seller_name,seller_url,title,url,source_site,offre,ingest_ts
0,"{'Etat': 'À rénover', 'Location Type': 't', 'L...",Immobilier Maroc > Immobilier Asilah > Maisons...,"[Immobilier Maroc, Immobilier Asilah, Maisons ...","Maison, à vendre",Prix 200 000 DH. À rénover. Superficie de 40 m...,[],8257931,[https://www.mubawab-media.com/ad/8/257/931F/h...,Asilah,200 000 DH,2025-11-25 00:14:41,2025-11-25T00:14:41.991240,False,,,Trouvez votre maison à acheter à Asilah. 1 Pièce,https://www.mubawab.ma/fr/a/8257931/trouvez-vo...,mubawab,vente,2025-11-25T00:17:42.038289
1,"{'Location ID': '2271', 'Etat': 'Nouveau', 'Pi...",Immobilier Maroc > Immobilier Marrakech > Appa...,"[Immobilier Maroc, Immobilier Marrakech, Appar...","Appartement, à vendre",Découvrez cet appartement à vendre. Prix 15 00...,"[Terrasse, Garage, Ascenseur, Piscine, Concier...",8257932,[https://www.mubawab-media.com/ad/8/257/932F/h...,Hivernage à\n\t\t\t\t\t\t\tMarrakech,15 000 000 DH,2025-11-25 00:14:45,2025-11-25T00:14:45.511966,True,AGENCE IMMOBILIERE STARLOGE,https://www.mubawab.ma/fr/b/11289/agence-immob...,Bel appartement à vendre à Hivernage. \n1 Pièc...,https://www.mubawab.ma/fr/a/8257932/bel-appart...,mubawab,vente,2025-11-25T00:17:42.038289


In [83]:
bronze_mub = df.select(
    # Identifiers
    "id",
    "url",
    "source_site",
    "offre",

    # Main business content
    "title",
    "description",
    "price_text",
    "location",

    # Dates
    "published_date",
    "scraping_time",
    "ingest_ts",

    # Metadata
    "category_label",
    "breadcrumbs",
    "breadcrumbs_list",
    "attributes",

    # Media
    "images",
    "equipments",

    # Seller info
    "seller_name",
    "seller_url",
    "seller_is_store"
)

bronze_mub.printSchema()

root
 |-- id: string (nullable = true)
 |-- url: string (nullable = true)
 |-- source_site: string (nullable = true)
 |-- offre: string (nullable = true)
 |-- title: string (nullable = true)
 |-- description: string (nullable = true)
 |-- price_text: string (nullable = true)
 |-- location: string (nullable = true)
 |-- published_date: string (nullable = true)
 |-- scraping_time: string (nullable = true)
 |-- ingest_ts: string (nullable = true)
 |-- category_label: string (nullable = true)
 |-- breadcrumbs: string (nullable = true)
 |-- breadcrumbs_list: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- attributes: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- images: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- equipments: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- seller_name: string (nullable = true)
 |-- seller_url: string (nullable = tru

In [84]:
from pyspark.sql import functions as F
bronze_mub.filter(
    (F.col("id").isNull()) | (F.col("url").isNull())
).count()


0

In [85]:
bronze_mub.groupBy("url").count().filter("count > 1").show()

+---+-----+
|url|count|
+---+-----+
+---+-----+



In [86]:
from pyspark.sql import Window
from pyspark.sql import functions as F

w = Window.partitionBy("url").orderBy(F.col("scraping_time").asc())

bronze_mub = (
    bronze_mub
    .withColumn("rn", F.row_number().over(w))
    .filter("rn = 1")
    .drop("rn")
)

In [87]:
bronze_mub.filter(F.col("source_site").isNull()).count()

0

In [89]:
bronze_mub.groupBy("offre") \
    .count() \
    .orderBy("count", ascending=False) \
    .show(truncate=False)

+--------+-----+
|offre   |count|
+--------+-----+
|location|33   |
|vente   |33   |
+--------+-----+



In [91]:
bronze_mub.select("price_text").show(20, truncate=False)

+-------------+
|price_text   |
+-------------+
|15 000 DH    |
|5 000 DH     |
|8 500 DH     |
|13 000 DH    |
|8 000 DH     |
|13 500 DH    |
|30 000 DH    |
|11 500 DH    |
|10 000 DH    |
|8 000 DH     |
|12 000 DH    |
|28 000 DH    |
|17 000 DH    |
|6 000 DH     |
|200 000 DH   |
|15 000 000 DH|
|6 600 DH     |
|5 500 DH     |
|35 000 000 DH|
|17 500 000 DH|
+-------------+
only showing top 20 rows



In [92]:
from pyspark.sql import functions as F

bronze_mub = (
    bronze_mub
    # 1) Créer la colonne numérique "price" à partir de "price_text"
    .withColumn(
        "price",
        F.when(
            # NULL, "null", chaîne vide → valeur nulle
            (F.col("price_text").isNull()) |
            (F.lower(F.col("price_text")) == "null") |
            (F.trim(F.col("price_text")) == ""),
            None
        ).otherwise(
            F.regexp_replace(                      # garder uniquement les chiffres
                F.regexp_replace(                  # enlever "DH" (insensible à la casse)
                    F.col("price_text"),
                    r"(?i)\s*dh\s*", ""
                ),
                r"[^\d]", ""                       # enlever tous les caractères non numériques (espaces, etc.)
            ).cast("double")
        )
    )
    # 2) Supprimer l’ancienne colonne texte
    .drop("price_text")
)

In [93]:
bronze_mub.select("price").show(20, truncate=False)

+--------+
|price   |
+--------+
|15000.0 |
|5000.0  |
|8500.0  |
|13000.0 |
|8000.0  |
|13500.0 |
|30000.0 |
|11500.0 |
|10000.0 |
|8000.0  |
|12000.0 |
|28000.0 |
|17000.0 |
|6000.0  |
|200000.0|
|1.5E7   |
|6600.0  |
|5500.0  |
|3.5E7   |
|1.75E7  |
+--------+
only showing top 20 rows



In [94]:
from pyspark.sql.types import DecimalType

bronze_mub = (
    bronze_mub
    .withColumn(
        "price",
        F.when(
            (F.col("price").isNull()),
            None
        ).otherwise(
            F.col("price").cast(DecimalType(20,0))
        )
    )
)

In [96]:
bronze_mub.select("location").show(10, truncate=False)

+-------------------------------------------------------+
|location                                               |
+-------------------------------------------------------+
|Casablanca Finance City à\n\t\t\t\t\t\t\tCasablanca    |
|Ouled mimoun à\n\t\t\t\t\t\t\tNador                    |
|Ferme Bretonne (Hay Arraha) à\n\t\t\t\t\t\t\tCasablanca|
|Casablanca Finance City à\n\t\t\t\t\t\t\tCasablanca    |
|Les princesses à\n\t\t\t\t\t\t\tCasablanca             |
|Casablanca Finance City à\n\t\t\t\t\t\t\tCasablanca    |
|Casablanca Finance City à\n\t\t\t\t\t\t\tCasablanca    |
|Racine à\n\t\t\t\t\t\t\tCasablanca                     |
|Casablanca Finance City à\n\t\t\t\t\t\t\tCasablanca    |
|Samlalia à\n\t\t\t\t\t\t\tMarrakech                    |
+-------------------------------------------------------+
only showing top 10 rows



In [97]:
from pyspark.sql import functions as F

bronze_mub = bronze_mub.withColumn(
    "location",
    F.trim(
        F.regexp_replace(
            F.col("location"),
            r"\s+",                # any whitespace (space, tab, newline)
            " "
        )
    )
)

bronze_mub.select("location").show(10, truncate=False)

+----------------------------------------+
|location                                |
+----------------------------------------+
|Casablanca Finance City à Casablanca    |
|Ouled mimoun à Nador                    |
|Ferme Bretonne (Hay Arraha) à Casablanca|
|Casablanca Finance City à Casablanca    |
|Les princesses à Casablanca             |
|Casablanca Finance City à Casablanca    |
|Casablanca Finance City à Casablanca    |
|Racine à Casablanca                     |
|Casablanca Finance City à Casablanca    |
|Samlalia à Marrakech                    |
+----------------------------------------+
only showing top 10 rows



In [98]:
bronze_mub.printSchema()

root
 |-- id: string (nullable = true)
 |-- url: string (nullable = true)
 |-- source_site: string (nullable = true)
 |-- offre: string (nullable = true)
 |-- title: string (nullable = true)
 |-- description: string (nullable = true)
 |-- location: string (nullable = true)
 |-- published_date: string (nullable = true)
 |-- scraping_time: string (nullable = true)
 |-- ingest_ts: string (nullable = true)
 |-- category_label: string (nullable = true)
 |-- breadcrumbs: string (nullable = true)
 |-- breadcrumbs_list: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- attributes: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- images: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- equipments: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- seller_name: string (nullable = true)
 |-- seller_url: string (nullable = true)
 |-- seller_is_store: boolean (nullable

In [99]:
bronze_mub.select("breadcrumbs").show(5, truncate=False)

+------------------------------------------------------------------------------------------------+
|breadcrumbs                                                                                     |
+------------------------------------------------------------------------------------------------+
|Immobilier Maroc > Immobilier Casablanca > Appartements Casablanca > Casablanca Finance City    |
|Immobilier Maroc > Immobilier Nador > Bureaux Nador > Ouled mimoun                              |
|Immobilier Maroc > Immobilier Casablanca > Appartements Casablanca > Ferme Bretonne (Hay Arraha)|
|Immobilier Maroc > Immobilier Casablanca > Bureaux Casablanca > Casablanca Finance City         |
|Immobilier Maroc > Immobilier Casablanca > Appartements Casablanca > Les princesses             |
+------------------------------------------------------------------------------------------------+
only showing top 5 rows



In [100]:
bronze_mub.select("breadcrumbs_list").show(5, truncate=False)

+-----------------------------------------------------------------------------------------------+
|breadcrumbs_list                                                                               |
+-----------------------------------------------------------------------------------------------+
|[Immobilier Maroc, Immobilier Casablanca, Appartements Casablanca, Casablanca Finance City]    |
|[Immobilier Maroc, Immobilier Nador, Bureaux Nador, Ouled mimoun]                              |
|[Immobilier Maroc, Immobilier Casablanca, Appartements Casablanca, Ferme Bretonne (Hay Arraha)]|
|[Immobilier Maroc, Immobilier Casablanca, Bureaux Casablanca, Casablanca Finance City]         |
|[Immobilier Maroc, Immobilier Casablanca, Appartements Casablanca, Les princesses]             |
+-----------------------------------------------------------------------------------------------+
only showing top 5 rows



In [102]:
bronze_mub.filter(F.col("breadcrumbs_list").isNull()).count()

0

In [105]:
bronze_mub = bronze_mub.drop("breadcrumbs")

In [107]:
bronze_mub.select("seller_name").show(5, truncate=False)

+-----------------------+
|seller_name            |
+-----------------------+
|Feel Home              |
|                       |
|HOMY real estate agency|
|Immo & Immo            |
|L'immobilière Exclusive|
+-----------------------+
only showing top 5 rows



In [108]:
bronze_mub.groupBy("seller_is_store").count().orderBy("count", ascending=False).show(truncate=False)

+---------------+-----+
|seller_is_store|count|
+---------------+-----+
|false          |63   |
|true           |3    |
+---------------+-----+



In [110]:
bronze_mub = bronze_mub.drop("seller_is_store")

In [111]:
bronze_mub.select("category_label").show(5, truncate=False)

+--------------------+
|category_label      |
+--------------------+
|Appartement         |
|Bureau, à louer     |
|Appartement, à louer|
|Bureau, à louer     |
|Appartement, à louer|
+--------------------+
only showing top 5 rows



In [113]:
bronze_mub = bronze_mub.drop("scraping_time")

In [114]:
bronze_mub = (
    bronze_mub
    # published_date : ex "2025-11-24T16:29:38.000Z"
    .withColumn(
        "published_date",
        F.to_timestamp("published_date", "yyyy-MM-dd'T'HH:mm:ss.SSS'Z'")
    )
    # ingest_ts : ex "2025-11-24 20:11:37.866812"
    .withColumn(
        "ingest_ts",
        F.to_timestamp("ingest_ts", "yyyy-MM-dd HH:mm:ss.SSSSSS")
    )
)

In [116]:
bronze_mub.printSchema()

root
 |-- id: string (nullable = true)
 |-- url: string (nullable = true)
 |-- source_site: string (nullable = true)
 |-- offre: string (nullable = true)
 |-- title: string (nullable = true)
 |-- description: string (nullable = true)
 |-- location: string (nullable = true)
 |-- published_date: timestamp (nullable = true)
 |-- ingest_ts: timestamp (nullable = true)
 |-- category_label: string (nullable = true)
 |-- breadcrumbs_list: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- attributes: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- images: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- equipments: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- seller_name: string (nullable = true)
 |-- seller_url: string (nullable = true)
 |-- price: decimal(20,0) (nullable = true)

